# Model Performance Evaluation

- R2 of all biomarkers for all models
- Sensitive analysis of model performance
- Relative importance of biomarkers
- ICC of all biomarkers for all models 
- Label efficiency analysis

> Note: As the bootstrap estimation of R2 and relative importance analysis based on bootsrap estimation, so the results are not very stable as the paper reported. 

In [26]:
import pandas as pd
from pathlib import Path
from config import *
from utils import rsquareCI, rsquareCI_v2, R2difference
import numpy as np

In [2]:
labels = pd.read_feather(Path("Raw") / "ClinicalLabels.feather")
# replace sex with Male =>1, Female => 0
labels["Sex"] = labels["Sex"].map({"Male": 1, "Female": 0})
labels

,eid,DataSetName,AG,ALP,ALT,APTT,AST,Alb,BMI,BNP,...,UA,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex
0,f412dfebe79027cff3ec509fb96cf74f,DX-EV,NaN,81.0,41.0,32.9,26.0,47.9,23.671254,45.0,...,546.0,7.50,7.66,75.0,1.52,NaN,NaN,NaN,60.0,1
1,d2b336e0e1a1ea5c15f5cec828589dac,DX-EV,NaN,96.0,19.0,33.3,21.0,45.2,17.146776,50.0,...,276.0,4.80,5.37,45.0,1.10,NaN,NaN,NaN,69.0,0
2,fdfe3a7ba64150efdfeb89d83cb1ccc0,DX-EV,NaN,54.0,15.0,32.6,16.0,42.3,27.458654,252.0,...,601.0,6.10,6.58,87.0,1.19,NaN,NaN,NaN,64.0,1
3,37aba8afa41165a0ad0675b7055d227e,DX-EV,NaN,62.0,13.0,35.0,14.0,34.6,23.120624,97.0,...,269.0,4.70,7.65,70.0,0.67,NaN,NaN,NaN,74.0,1
4,118f24f5284b781e354c37409858f59a,DX-EV,NaN,66.0,20.0,37.4,16.0,45.2,28.577961,61.0,...,344.0,4.60,5.36,75.0,1.40,NaN,NaN,NaN,60.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9765,1f5b5321a421d4a566f6c41262ba774e,APP-EV,1.43,95.4,15.3,NaN,18.6,42.4,19.948060,NaN,...,265.5,4.77,NaN,53.0,0.30,NaN,1.72,0.41,60.0,0
9766,234d3b32bb21791791da8741b6dd072b,APP-EV,0.98,100.5,23.3,NaN,28.8,NaN,27.942143,NaN,...,308.1,5.99,NaN,68.0,5.40,NaN,0.86,0.17,72.0,0
9767,9b83d1551a89149543cdd635f8f2690c,APP-EV,1.38,NaN,14.9,NaN,21.9,NaN,19.628265,NaN,...,NaN,5.36,NaN,49.0,0.30,NaN,2.73,NaN,59.0,0
9768,e0b7bca81e873378e05ddf98f369d4ed,APP-EV,1.53,NaN,28.9,NaN,26.2,NaN,32.466181,NaN,...,308.3,5.43,NaN,78.0,4.60,NaN,2.68,NaN,69.0,0


In [3]:
# load data

totalDataDict = {
    "Ummae": pd.read_feather(Path("Raw") / "Ummae.feather").merge(
        labels, on=["eid", "DataSetName"], suffixes=["_Ummae", ""]
    ),
    "ResNet18": pd.read_feather(Path("Raw") / "ResNet18.feather").merge(
        labels, on=["eid", "DataSetName"], suffixes=["_ResNet18", ""]
    ),
    "SwinLarge": pd.read_feather(Path("Raw") / "SwinLarge.feather").merge(
        labels, on=["eid", "DataSetName"], suffixes=["_SwinLarge", ""]
    ),
}

In [ ]:
# save this data for later
import pickle
pickle.dump(totalDataDict, open(RawDataDir / "totalDataDict.pkl", "wb"))

In [5]:
exclude_list = ["train", "validation"]

centers = [
    "AZ-TR",
    "AZ-EV1",
    "AZ-EV2",
    "AZTZ-EV",
    "DX-EV",
]
models = ["Ummae", "SwinLarge", "ResNet18"]

## Calulate R2 of Deep Learning Model



In [6]:
## cal qt
from ppp_prediction.metrics import cal_qt_metrics
from ppp_prediction.corr_v3 import cal_corr_v3

qt_metrics_with_cov = []
# cov = ['Age','Sex', 'Height', 'Weight', 'BMI']
mean_data = True  # mean data or not; For our job, use mean of one more images to get facial prediction.
for model, df in totalDataDict.items():
    for qt in qts:
        # for qt in ["BMI", "Height", "Weight"]:
        # for center in ['DX-EV']:
        for center in centers:
            # if qt in cov:
            #     continue

            model_col = f"{qt}_{model}"
            if model_col not in df.columns or qt not in df.columns:
                print(f"{model_col} or {qt} not in data at {center}")
                continue
            to_cal_data = (
                df.query("DataSetName == @center and type not in @exclude_list ")
                .copy()[["eid", model_col, qt]]
                .dropna()
            )
            to_cal_data[qt] = to_cal_data[qt].astype(float)
            to_cal_data[model_col] = to_cal_data[model_col].astype(float)

            if mean_data:
                to_cal_data = to_cal_data.groupby("eid").mean().reset_index()
            if to_cal_data.shape[0] == 0:
                print(f"{model_col} or {qt} not in data at {center}")
                continue
            else:
                print(
                    f"Start to calculate metrics for {qt} and {model_col} at {center}"
                )

            metrics_qt = cal_corr_v3(
                to_cal_data,
                x=f"{qt}_{model}",
                y=qt,
                norm_x=False,
                model_type="glm",
            )
            metrics_qt_2 = cal_qt_metrics(
                to_cal_data[qt], to_cal_data[f"{qt}_{model}"], ci=True, n_resamples=1000
            )
            for key, values in metrics_qt_2.items():
                metrics_qt[key] = values
            metrics_qt["Label"] = qt
            metrics_qt["Model"] = model
            metrics_qt["Center"] = center

            qt_metrics_with_cov.append(metrics_qt)


metrics_qt_with_cov_df = pd.concat(qt_metrics_with_cov)

metrics_qt_with_cov_df

Start to calculate metrics for AG and AG_Ummae at AZ-TR
threads is 4 and x is AG_Ummae and y is AG, which is str, so don't supported for multi threads
AG ~ AG_Ummae
Start to calculate metrics for AG and AG_Ummae at AZ-EV1
threads is 4 and x is AG_Ummae and y is AG, which is str, so don't supported for multi threads
AG ~ AG_Ummae
Start to calculate metrics for AG and AG_Ummae at AZ-EV2
threads is 4 and x is AG_Ummae and y is AG, which is str, so don't supported for multi threads
AG ~ AG_Ummae
AG_Ummae or AG not in data at AZTZ-EV
AG_Ummae or AG not in data at DX-EV
Start to calculate metrics for ALP and ALP_Ummae at AZ-TR
threads is 4 and x is ALP_Ummae and y is ALP, which is str, so don't supported for multi threads
ALP ~ ALP_Ummae
Start to calculate metrics for ALP and ALP_Ummae at AZ-EV1
threads is 4 and x is ALP_Ummae and y is ALP, which is str, so don't supported for multi threads
ALP ~ ALP_Ummae
Start to calculate metrics for ALP and ALP_Ummae at AZ-EV2
threads is 4 and x is ALP_U

,x,y,cov,var,coef,std,z,pvalue,lower_ci,upper_ci,...,p_pearsonr_score,spearmanr_LCI,spearmanr_UCI,p_spearmanr_score,R2_from_R,R2_from_R_LCI,R2_from_R_UCI,Label,Model,Center
0,AG_Ummae,AG,,AG_Ummae,0.005770,0.010877,0.530446,5.958029e-01,-0.015549,0.027088,...,5.960589e-01,-0.030619,0.087022,4.550583e-01,0.000611,-0.003657,0.003531,AG,Ummae,AZ-TR
0,AG_Ummae,AG,,AG_Ummae,-0.190210,0.115329,-1.649279,9.909045e-02,-0.416250,0.035831,...,9.930382e-02,-0.009661,0.060763,4.134248e-01,-0.001845,-0.003678,0.002177,AG,Ummae,AZ-EV1
0,AG_Ummae,AG,,AG_Ummae,0.245602,0.190392,1.289982,1.970569e-01,-0.127559,0.618763,...,1.971963e-01,0.041393,0.086661,7.511834e-04,0.000777,-0.002559,0.002070,AG,Ummae,AZ-EV2
0,ALP_Ummae,ALP,,ALP_Ummae,0.871012,0.404204,2.154881,3.117112e-02,0.078786,1.663238,...,3.168947e-02,0.131147,0.229620,6.847733e-04,0.009994,-0.026293,0.017190,ALP,Ummae,AZ-TR
0,ALP_Ummae,ALP,,ALP_Ummae,1.077639,0.194811,5.531711,3.171227e-08,0.695816,1.459461,...,3.923919e-08,0.088851,0.188230,5.973532e-07,0.025919,-0.000539,0.040655,ALP,Ummae,AZ-EV1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Age_SwinLarge,Age,,Age_SwinLarge,1.043968,0.018488,56.467222,0.000000e+00,1.007732,1.080204,...,1.312120e-213,0.919141,0.944047,3.029327e-208,0.869632,0.860105,0.879849,Age,SwinLarge,AZ-TR
0,Age_SwinLarge,Age,,Age_SwinLarge,1.067418,0.010948,97.494699,0.000000e+00,1.045959,1.088876,...,0.000000e+00,0.916104,0.926347,0.000000e+00,0.864015,0.851810,0.871977,Age,SwinLarge,AZ-EV1
0,Age_SwinLarge,Age,,Age_SwinLarge,1.055055,0.009493,111.139859,0.000000e+00,1.036449,1.073661,...,0.000000e+00,0.909125,0.917295,0.000000e+00,0.846202,0.840629,0.855662,Age,SwinLarge,AZ-EV2
0,Age_SwinLarge,Age,,Age_SwinLarge,1.016621,0.035551,28.596063,7.520552e-180,0.946942,1.086300,...,2.655911e-73,0.862708,0.939229,5.077124e-70,0.800340,0.771293,0.828639,Age,SwinLarge,AZTZ-EV


In [7]:
metrics_qt_with_cov_df["mlog10p"] = (
    -np.log10(metrics_qt_with_cov_df["pvalue"])
).replace(np.inf, 308)

metrics_qt_with_cov_df

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,x,y,cov,var,coef,std,z,pvalue,lower_ci,upper_ci,...,spearmanr_LCI,spearmanr_UCI,p_spearmanr_score,R2_from_R,R2_from_R_LCI,R2_from_R_UCI,Label,Model,Center,mlog10p
0,AG_Ummae,AG,,AG_Ummae,0.005770,0.010877,0.530446,5.958029e-01,-0.015549,0.027088,...,-0.030619,0.087022,4.550583e-01,0.000611,-0.003657,0.003531,AG,Ummae,AZ-TR,0.224897
0,AG_Ummae,AG,,AG_Ummae,-0.190210,0.115329,-1.649279,9.909045e-02,-0.416250,0.035831,...,-0.009661,0.060763,4.134248e-01,-0.001845,-0.003678,0.002177,AG,Ummae,AZ-EV1,1.003968
0,AG_Ummae,AG,,AG_Ummae,0.245602,0.190392,1.289982,1.970569e-01,-0.127559,0.618763,...,0.041393,0.086661,7.511834e-04,0.000777,-0.002559,0.002070,AG,Ummae,AZ-EV2,0.705408
0,ALP_Ummae,ALP,,ALP_Ummae,0.871012,0.404204,2.154881,3.117112e-02,0.078786,1.663238,...,0.131147,0.229620,6.847733e-04,0.009994,-0.026293,0.017190,ALP,Ummae,AZ-TR,1.506248
0,ALP_Ummae,ALP,,ALP_Ummae,1.077639,0.194811,5.531711,3.171227e-08,0.695816,1.459461,...,0.088851,0.188230,5.973532e-07,0.025919,-0.000539,0.040655,ALP,Ummae,AZ-EV1,7.498773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Age_SwinLarge,Age,,Age_SwinLarge,1.043968,0.018488,56.467222,0.000000e+00,1.007732,1.080204,...,0.919141,0.944047,3.029327e-208,0.869632,0.860105,0.879849,Age,SwinLarge,AZ-TR,308.000000
0,Age_SwinLarge,Age,,Age_SwinLarge,1.067418,0.010948,97.494699,0.000000e+00,1.045959,1.088876,...,0.916104,0.926347,0.000000e+00,0.864015,0.851810,0.871977,Age,SwinLarge,AZ-EV1,308.000000
0,Age_SwinLarge,Age,,Age_SwinLarge,1.055055,0.009493,111.139859,0.000000e+00,1.036449,1.073661,...,0.909125,0.917295,0.000000e+00,0.846202,0.840629,0.855662,Age,SwinLarge,AZ-EV2,308.000000
0,Age_SwinLarge,Age,,Age_SwinLarge,1.016621,0.035551,28.596063,7.520552e-180,0.946942,1.086300,...,0.862708,0.939229,5.077124e-70,0.800340,0.771293,0.828639,Age,SwinLarge,AZTZ-EV,179.123750


In [8]:
metrics_qt_with_cov_df

,x,y,cov,var,coef,std,z,pvalue,lower_ci,upper_ci,...,spearmanr_LCI,spearmanr_UCI,p_spearmanr_score,R2_from_R,R2_from_R_LCI,R2_from_R_UCI,Label,Model,Center,mlog10p
0,AG_Ummae,AG,,AG_Ummae,0.005770,0.010877,0.530446,5.958029e-01,-0.015549,0.027088,...,-0.030619,0.087022,4.550583e-01,0.000611,-0.003657,0.003531,AG,Ummae,AZ-TR,0.224897
0,AG_Ummae,AG,,AG_Ummae,-0.190210,0.115329,-1.649279,9.909045e-02,-0.416250,0.035831,...,-0.009661,0.060763,4.134248e-01,-0.001845,-0.003678,0.002177,AG,Ummae,AZ-EV1,1.003968
0,AG_Ummae,AG,,AG_Ummae,0.245602,0.190392,1.289982,1.970569e-01,-0.127559,0.618763,...,0.041393,0.086661,7.511834e-04,0.000777,-0.002559,0.002070,AG,Ummae,AZ-EV2,0.705408
0,ALP_Ummae,ALP,,ALP_Ummae,0.871012,0.404204,2.154881,3.117112e-02,0.078786,1.663238,...,0.131147,0.229620,6.847733e-04,0.009994,-0.026293,0.017190,ALP,Ummae,AZ-TR,1.506248
0,ALP_Ummae,ALP,,ALP_Ummae,1.077639,0.194811,5.531711,3.171227e-08,0.695816,1.459461,...,0.088851,0.188230,5.973532e-07,0.025919,-0.000539,0.040655,ALP,Ummae,AZ-EV1,7.498773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Age_SwinLarge,Age,,Age_SwinLarge,1.043968,0.018488,56.467222,0.000000e+00,1.007732,1.080204,...,0.919141,0.944047,3.029327e-208,0.869632,0.860105,0.879849,Age,SwinLarge,AZ-TR,308.000000
0,Age_SwinLarge,Age,,Age_SwinLarge,1.067418,0.010948,97.494699,0.000000e+00,1.045959,1.088876,...,0.916104,0.926347,0.000000e+00,0.864015,0.851810,0.871977,Age,SwinLarge,AZ-EV1,308.000000
0,Age_SwinLarge,Age,,Age_SwinLarge,1.055055,0.009493,111.139859,0.000000e+00,1.036449,1.073661,...,0.909125,0.917295,0.000000e+00,0.846202,0.840629,0.855662,Age,SwinLarge,AZ-EV2,308.000000
0,Age_SwinLarge,Age,,Age_SwinLarge,1.016621,0.035551,28.596063,7.520552e-180,0.946942,1.086300,...,0.862708,0.939229,5.077124e-70,0.800340,0.771293,0.828639,Age,SwinLarge,AZTZ-EV,179.123750


In [9]:
metrics_qt_with_cov_df.query(" pvalue <0.05").sort_values(
    "r2_score", ascending=False
).groupby(["Center", "Label"]).head(1)["Model"].value_counts()

Ummae        157
SwinLarge     62
ResNet18      18
Name: Model, dtype: int64

In [10]:
from ppp_prediction.corr_v3 import generate_multipletests_result

# add fdr_bh
metrics_qt_with_cov_df_fdr = (
    generate_multipletests_result(metrics_qt_with_cov_df, method="fdr_bh")
    .sort_values("r2_score", ascending=False)
    .rename(columns={"r2_score": "R2"})
)

# add se, lci and uci
se, lci, uci = rsquareCI_v2(
    metrics_qt_with_cov_df_fdr["R2"], metrics_qt_with_cov_df_fdr["N"], 1
)
metrics_qt_with_cov_df_fdr["R2_SE"] = se
metrics_qt_with_cov_df_fdr["R2_LCI"] = lci
metrics_qt_with_cov_df_fdr["R2_UCI"] = uci
metrics_qt_with_cov_df_fdr

,x,y,cov,var,coef,std,z,pvalue,lower_ci,upper_ci,...,R2_from_R_UCI,Label,Model,Center,mlog10p,p_adj,reject,R2_SE,R2_LCI,R2_UCI
0,Age_Ummae,Age,,Age_Ummae,1.004532,0.017506,57.381750,0.000000,0.970221,1.038843,...,0.882198,Age,Ummae,AZ-TR,308.000000,0.000000,True,0.010735,0.851761,0.894703
0,Age_SwinLarge,Age,,Age_SwinLarge,1.043968,0.018488,56.467222,0.000000,1.007732,1.080204,...,0.879849,Age,SwinLarge,AZ-TR,308.000000,0.000000,True,0.011017,0.847597,0.891667
0,Age_SwinLarge,Age,,Age_SwinLarge,1.067418,0.010948,97.494699,0.000000,1.045959,1.088876,...,0.871977,Age,SwinLarge,AZ-EV1,308.000000,0.000000,True,0.006516,0.850982,0.877048
0,Age_Ummae,Age,,Age_Ummae,0.999671,0.010303,97.024918,0.000000,0.979477,1.019865,...,0.871181,Age,Ummae,AZ-EV1,308.000000,0.000000,True,0.006567,0.849743,0.876009
0,Age_ResNet18,Age,,Age_ResNet18,1.001261,0.010407,96.207629,0.000000,0.980863,1.021659,...,0.866434,Age,ResNet18,AZ-EV1,308.000000,0.000000,True,0.006655,0.847551,0.874173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,K_SwinLarge,K,,K_SwinLarge,-0.010643,0.403425,-0.026382,0.978952,-0.801342,0.780055,...,0.000016,K,SwinLarge,AZ-TR,0.009238,0.981314,False,0.000114,-0.000226,0.000229
0,sdLDL_ResNet18,sdLDL,,sdLDL_ResNet18,0.016305,1.808151,0.009018,0.992805,-3.527606,3.560216,...,0.048347,sdLDL,ResNet18,AZTZ-EV,0.003136,0.994001,False,0.000193,-0.000385,0.000387
0,hs-CRP_SwinLarge,hs-CRP,,hs-CRP_SwinLarge,-0.039495,1.420691,-0.027800,0.977822,-2.823998,2.745009,...,0.002647,hs-CRP,SwinLarge,AZ-EV1,0.009740,0.981314,False,0.000049,-0.000098,0.000100
0,T3_ResNet18,T3,,T3_ResNet18,-0.001724,0.052123,-0.033085,0.973607,-0.103884,0.100435,...,0.000566,T3,ResNet18,AZ-EV2,0.011616,0.979500,False,0.000030,-0.000059,0.000060


In [11]:
from ppp_prediction.metrics.utils import format_metrics

add_metric = ["R2", "spearmanr", "pearsonr"]

for metric in add_metric:

    metrics_qt_with_cov_df_fdr[f"{metric} (95% CI)"] = format_metrics(
        metrics_qt_with_cov_df_fdr[f"{metric}"],
        x_lci=metrics_qt_with_cov_df_fdr[f"{metric}_LCI"],
        x_uci=metrics_qt_with_cov_df_fdr[f"{metric}_UCI"],
        round=3,
        data=metrics_qt_with_cov_df_fdr,
    )

metrics_qt_with_cov_df_fdr = metrics_qt_with_cov_df_fdr.reset_index(drop=True)[
    [
        "Model",
        "Label",
        "Center",
        "N",
        "spearmanr",
        "pearsonr",
        "p_adj",
        "pvalue",
        "R2",
        "R2_LCI",
        "R2_UCI",
        "R2 (95% CI)",
        "spearmanr_LCI",
        "spearmanr_UCI",
        "spearmanr (95% CI)",
        "pearsonr_LCI",
        "pearsonr_UCI",
        "pearsonr (95% CI)",
    ]
]
metrics_qt_with_cov_df_fdr.sort_values(by=["Center", "Model", "Label"], inplace=True)
metrics_qt_with_cov_df_fdr = metrics_qt_with_cov_df_fdr.reset_index(drop=True)
metrics_qt_with_cov_df_fdr.to_csv(RawDataDir / "metrics_qt.csv")
metrics_qt_with_cov_df_fdr.to_feather(RawDataDir / "metrics_qt.feather")
metrics_qt_with_cov_df_fdr

,Model,Label,Center,N,spearmanr,pearsonr,p_adj,pvalue,R2,R2_LCI,R2_UCI,R2 (95% CI),spearmanr_LCI,spearmanr_UCI,spearmanr (95% CI),pearsonr_LCI,pearsonr_UCI,pearsonr (95% CI)
0,ResNet18,AG,AZ-EV1,1474,0.074119,-0.007757,8.067738e-01,7.659982e-01,0.000060,-0.000746,0.000866,"0.000 (-0.001, 0.001)",0.036330,0.096290,"0.074 (0.036, 0.096)",-0.053324,0.040154,"-0.008 (-0.053, 0.040)"
1,ResNet18,ALP,AZ-EV1,1152,0.120766,0.142960,2.376341e-06,9.665502e-07,0.020438,0.003984,0.036891,"0.020 (0.004, 0.037)",0.072613,0.155722,"0.121 (0.073, 0.156)",0.106501,0.187912,"0.143 (0.107, 0.188)"
2,ResNet18,ALT,AZ-EV1,1495,0.260491,0.203048,5.335247e-15,1.123548e-15,0.041228,0.021136,0.061321,"0.041 (0.021, 0.061)",0.246248,0.301909,"0.260 (0.246, 0.302)",0.119427,0.229035,"0.203 (0.119, 0.229)"
3,ResNet18,APTT,AZ-EV1,1495,0.044554,0.036771,1.988947e-01,1.550948e-01,0.001352,-0.002438,0.005142,"0.001 (-0.002, 0.005)",0.016075,0.097595,"0.045 (0.016, 0.098)",-0.009976,0.067058,"0.037 (-0.010, 0.067)"
4,ResNet18,AST,AZ-EV1,1495,-0.007178,0.044670,1.146657e-01,8.403301e-02,0.001995,-0.002606,0.006597,"0.002 (-0.003, 0.007)",-0.035225,0.032541,"-0.007 (-0.035, 0.033)",0.006875,0.082360,"0.045 (0.007, 0.082)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,Ummae,Urea,DX-EV,895,0.189554,0.193474,1.176252e-08,3.793449e-09,0.037432,0.012629,0.062235,"0.037 (0.013, 0.062)",0.133096,0.235492,"0.190 (0.133, 0.235)",0.167554,0.252996,"0.193 (0.168, 0.253)"
827,Ummae,WBC,DX-EV,893,0.148479,0.139583,5.568552e-05,2.579895e-05,0.019483,0.001235,0.037732,"0.019 (0.001, 0.038)",0.102851,0.191618,"0.148 (0.103, 0.192)",0.085855,0.168623,"0.140 (0.086, 0.169)"
828,Ummae,Weight,DX-EV,800,0.682028,0.676797,2.055708e-147,1.063724e-148,0.458055,0.406409,0.509700,"0.458 (0.406, 0.510)",0.653147,0.719780,"0.682 (0.653, 0.720)",0.629848,0.738134,"0.677 (0.630, 0.738)"
829,Ummae,eGFR(CKD-EPI),DX-EV,895,0.474502,0.428795,1.041746e-44,1.153317e-45,0.183865,0.137257,0.230473,"0.184 (0.137, 0.230)",0.444578,0.515038,"0.475 (0.445, 0.515)",0.391157,0.480555,"0.429 (0.391, 0.481)"


##  Sensitivity Analysis with adjustment for covariates

- Adjusted for age, sex, height, weight and BMI
- Sensitive analysis in AZ-EV1

In [12]:
totalDataDict[model]

,eid,type,DataSetName,AG_SwinLarge,ALP_SwinLarge,ALT_SwinLarge,APTT_SwinLarge,AST_SwinLarge,Alb_SwinLarge,BMI_SwinLarge,...,UA,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex
0,f412dfebe79027cff3ec509fb96cf74f,test,DX-EV,16.108089,76.876686,23.046091,30.960651,20.551130,45.827717,25.464987,...,546.0,7.50,7.66,75.0,1.52,NaN,NaN,NaN,60.0,1
1,f412dfebe79027cff3ec509fb96cf74f,test,DX-EV,15.968744,80.820537,24.264486,31.017006,20.639469,45.601379,25.911572,...,546.0,7.50,7.66,75.0,1.52,NaN,NaN,NaN,60.0,1
2,f412dfebe79027cff3ec509fb96cf74f,test,DX-EV,15.961616,77.306035,24.111233,30.973713,20.608097,45.895817,24.399029,...,546.0,7.50,7.66,75.0,1.52,NaN,NaN,NaN,60.0,1
3,d2b336e0e1a1ea5c15f5cec828589dac,test,DX-EV,15.913592,80.152704,22.671471,31.560030,21.127054,45.140827,23.344477,...,276.0,4.80,5.37,45.0,1.10,NaN,NaN,NaN,69.0,0
4,d2b336e0e1a1ea5c15f5cec828589dac,test,DX-EV,15.537694,83.821594,23.183491,31.497517,21.238968,44.742924,23.571339,...,276.0,4.80,5.37,45.0,1.10,NaN,NaN,NaN,69.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71332,49741c348e2bd67f64e28ed8bb1c4382,test,AZTZ-EV,15.599611,85.053116,18.750061,30.208441,24.758336,45.985344,25.840652,...,338.8,6.91,7.82,75.0,3.14,322.6,NaN,0.76,73.0,0
71333,7ff193191191d45f5f0367d41a5d634d,test,AZTZ-EV,16.841013,83.995918,21.453663,31.294296,20.950385,45.559196,26.350403,...,293.0,3.03,4.10,84.0,0.32,0.6,NaN,NaN,54.0,1
71334,87d290fc32cf7c0dd04ba90744c8bb3c,test,AZTZ-EV,16.365156,76.282748,25.178137,31.774067,20.993694,45.778004,25.140323,...,255.7,6.98,5.51,65.0,NaN,0.0,NaN,NaN,53.0,1
71335,ddc60c6f4b0ca7c96aec0900d988d1c2,test,AZTZ-EV,15.299160,77.195816,30.249599,31.347206,20.526112,45.733543,24.165762,...,315.5,7.03,5.96,NaN,NaN,10.8,NaN,NaN,56.0,1


In [13]:
## cal qt
from ppp_prediction.corr_v3 import cal_corr_v3

qt_metrics_with_cov_2 = []
cov = ["Age", "Sex", "Height", "Weight", "BMI"]
mean_data = True  # mean data or not
# for model, df in totalDataDict.items():
for model in ["Ummae"]:
    df = totalDataDict[model]

    for qt in qts:
        # for qt in ["BMI", "Height", "Weight"]:
        for center in ["AZ-EV1"]:
            # for center in centers:
            if qt in cov:
                continue

            model_col = f"{qt}_{model}"
            if model_col not in df.columns or qt not in df.columns:
                print(f"{model_col} or {qt} not in data at {center}")
                continue
            to_cal_data = (
                df.query("DataSetName == @center and type not in @exclude_list ")
                .copy()[["eid", model_col, qt] + cov]
                .dropna()
            )
            to_cal_data[qt] = to_cal_data[qt].astype(float)
            to_cal_data[model_col] = to_cal_data[model_col].astype(float)

            if mean_data:
                to_cal_data = to_cal_data.groupby("eid").mean().reset_index()
            if to_cal_data.shape[0] == 0:
                print(f"{model_col} or {qt} not in data at {center}")
                continue
            else:
                print(
                    f"Start to calculate metrics for {qt} and {model_col} at {center}"
                )

            metrics_qt = cal_corr_v3(
                to_cal_data,
                x=f"{qt}_{model}",
                y=qt,
                norm_x=False,
                cov=cov,
                model_type="glm",
                adjust=True,
            )

            metrics_qt["Label"] = qt
            metrics_qt["Model"] = model
            metrics_qt["Center"] = center

            qt_metrics_with_cov_2.append(metrics_qt)


metrics_qt_with_cov_df_2 = pd.concat(qt_metrics_with_cov_2)

metrics_qt_with_cov_df_2

Start to calculate metrics for AG and AG_Ummae at AZ-EV1
threads is 4 and x is AG_Ummae and y is AG, which is str, so don't supported for multi threads
passed data have 1434 rows
used data have 1434 rows after dropna
AG ~ AG_Ummae
Start to calculate metrics for ALP and ALP_Ummae at AZ-EV1
threads is 4 and x is ALP_Ummae and y is ALP, which is str, so don't supported for multi threads
passed data have 1124 rows
used data have 1124 rows after dropna
ALP ~ ALP_Ummae
Start to calculate metrics for ALT and ALT_Ummae at AZ-EV1
threads is 4 and x is ALT_Ummae and y is ALT, which is str, so don't supported for multi threads
passed data have 1453 rows
used data have 1453 rows after dropna
ALT ~ ALT_Ummae
Start to calculate metrics for APTT and APTT_Ummae at AZ-EV1
threads is 4 and x is APTT_Ummae and y is APTT, which is str, so don't supported for multi threads
passed data have 1454 rows
used data have 1454 rows after dropna
APTT ~ APTT_Ummae
Start to calculate metrics for AST and AST_Ummae at 

,x,y,cov,var,coef,std,z,pvalue,lower_ci,upper_ci,...,n_control,pearsonr,spearmanr,explained_variance_score,r2_score,norm_x,adjust,Label,Model,Center
0,AG_Ummae,AG,Age + Sex + Height + Weight + BMI,AG_Ummae,-0.216095,0.118475,-1.823967,6.815709e-02,-0.448303,0.016112,...,-1270.470000,0.048144,0.042649,0.002318,0.002318,False,1,AG,Ummae,AZ-EV1
0,ALP_Ummae,ALP,Age + Sex + Height + Weight + BMI,ALP_Ummae,0.505219,0.196531,2.570676,1.015001e-02,0.120024,0.890413,...,-87760.683333,0.076520,0.056294,0.005855,0.005855,False,1,ALP,Ummae,AZ-EV1
0,ALT_Ummae,ALT,Age + Sex + Height + Weight + BMI,ALT_Ummae,0.121686,0.258287,0.471129,6.375483e-01,-0.384546,0.627919,...,-37369.066667,0.012367,-0.075021,0.000153,0.000153,False,1,ALT,Ummae,AZ-EV1
0,APTT_Ummae,APTT,Age + Sex + Height + Weight + BMI,APTT_Ummae,0.555269,0.407806,1.361600,1.733242e-01,-0.244017,1.354555,...,-43606.520000,0.035710,0.044507,0.001275,0.001275,False,1,APTT,Ummae,AZ-EV1
0,AST_Ummae,AST,Age + Sex + Height + Weight + BMI,AST_Ummae,-0.135310,1.649727,-0.082020,9.346310e-01,-3.368715,3.098095,...,-30971.433333,0.002153,-0.026442,0.000005,0.000005,False,1,AST,Ummae,AZ-EV1
0,Alb_Ummae,Alb,Age + Sex + Height + Weight + BMI,Alb_Ummae,0.652016,0.169241,3.852583,1.168785e-04,0.320309,0.983723,...,-46398.683333,0.114768,0.111242,0.013172,0.013172,False,1,Alb,Ummae,AZ-EV1
0,BNP_Ummae,BNP,Age + Sex + Height + Weight + BMI,BNP_Ummae,0.402356,0.126222,3.187693,1.434125e-03,0.154966,0.649746,...,-110055.350000,0.083708,-0.143195,0.007007,0.007007,False,1,BNP,Ummae,AZ-EV1
0,C1q_Ummae,C1q,Age + Sex + Height + Weight + BMI,C1q_Ummae,0.320443,0.255708,1.253162,2.101467e-01,-0.180735,0.821621,...,-177607.916667,0.039817,0.036735,0.001585,0.001585,False,1,C1q,Ummae,AZ-EV1
0,Cl_Ummae,Cl,Age + Sex + Height + Weight + BMI,Cl_Ummae,1.742686,2.219891,0.785032,4.324347e-01,-2.608220,6.093591,...,-149389.600000,0.020741,0.163601,0.000430,0.000430,False,1,Cl,Ummae,AZ-EV1
0,CK_Ummae,CK,Age + Sex + Height + Weight + BMI,CK_Ummae,0.030109,0.186581,0.161373,8.717994e-01,-0.335583,0.395801,...,-131089.416667,0.004244,-0.042385,0.000018,0.000018,False,1,CK,Ummae,AZ-EV1


In [14]:
from ppp_prediction.corr_v3 import generate_multipletests_result

metrics_qt_with_cov_df_2_fdr = (
    generate_multipletests_result(metrics_qt_with_cov_df_2, method="fdr_bh")
    .query("p_adj <0.05")  # only keep q-values < 0.05
    .sort_values("r2_score", ascending=False)[
        [
            "Model",
            "Label",
            "Center",
            "pearsonr",
            "spearmanr",
            "N",
            "r2_score",
            "p_adj",
            "pvalue",
        ]
    ]
    .rename(columns={"r2_score": "R2"})
)


metrics_qt_with_cov_df_2_fdr

,Model,Label,Center,pearsonr,spearmanr,N,R2,p_adj,pvalue
0,Ummae,HbA1C%,AZ-EV1,0.352716,0.322341,1429,0.124409,2.981030e-44,5.229877e-46
0,Ummae,TG,AZ-EV1,0.224667,0.106085,1450,0.050475,4.937820e-17,1.732568e-18
0,Ummae,Glu,AZ-EV1,0.189534,0.159546,1454,0.035923,3.612770e-12,1.901458e-13
0,Ummae,Hb,AZ-EV1,0.164079,0.163665,1451,0.026922,3.457753e-09,2.426493e-10
0,Ummae,LYM,AZ-EV1,0.151238,0.161752,1451,0.022873,6.551018e-08,5.746507e-09
0,Ummae,HDLC,AZ-EV1,0.144161,0.098044,1450,0.020782,2.814658e-07,2.962798e-08
0,Ummae,nonHDL,AZ-EV1,0.141756,0.121235,1406,0.020095,6.562744e-07,8.059510e-08
0,Ummae,sdLDL,AZ-EV1,0.141688,0.127835,1001,0.020076,2.882787e-05,6.069024e-06
0,Ummae,FFA,AZ-EV1,0.136201,0.114612,1000,0.018551,6.156390e-05,1.404089e-05
0,Ummae,WBC,AZ-EV1,0.127340,0.130535,1451,0.016216,7.291703e-06,1.023397e-06


In [15]:
metrics_qt_with_cov_df_2_fdr.sort_values("p_adj", ascending=True)

,Model,Label,Center,pearsonr,spearmanr,N,R2,p_adj,pvalue
0,Ummae,HbA1C%,AZ-EV1,0.352716,0.322341,1429,0.124409,2.981030e-44,5.229877e-46
0,Ummae,TG,AZ-EV1,0.224667,0.106085,1450,0.050475,4.937820e-17,1.732568e-18
0,Ummae,Glu,AZ-EV1,0.189534,0.159546,1454,0.035923,3.612770e-12,1.901458e-13
0,Ummae,Hb,AZ-EV1,0.164079,0.163665,1451,0.026922,3.457753e-09,2.426493e-10
0,Ummae,LYM,AZ-EV1,0.151238,0.161752,1451,0.022873,6.551018e-08,5.746507e-09
0,Ummae,HDLC,AZ-EV1,0.144161,0.098044,1450,0.020782,2.814658e-07,2.962798e-08
0,Ummae,nonHDL,AZ-EV1,0.141756,0.121235,1406,0.020095,6.562744e-07,8.059510e-08
0,Ummae,WBC,AZ-EV1,0.127340,0.130535,1451,0.016216,7.291703e-06,1.023397e-06
0,Ummae,NE%,AZ-EV1,0.127274,0.130653,1413,0.016199,9.090502e-06,1.435342e-06
0,Ummae,RBC,AZ-EV1,0.123878,0.116759,1451,0.015346,1.147351e-05,2.012896e-06


In [16]:
metrics_qt_with_cov_df_2_fdr.sort_values("Label")["Center"].value_counts()

AZ-EV1    26
Name: Center, dtype: int64

In [17]:
metrics_qt_with_cov_df_2_fdr.to_csv(RawDataDir / "adjusted_ummae_r2.csv", index=False)

## Relative Importance of Features

- calculate the relative importance of features by `relaimpo` 

In [18]:
from utils import get_cat_var_name, get_cat_var_subname, columnsFormatV1, calc_relimp

In [19]:
relativeDir = RawDataDir / "Ummae_Basic_Relative"
relativeDir.mkdir(exist_ok=True)
relative_res_list = []
for model in ["Ummae"]:
    df = totalDataDict[model]
    # replace sex

    for qt in metrics_qt_with_cov_df_2_fdr["Label"].tolist():

        train_data = df[[qt, f"{qt}_Ummae"] + cov].copy().dropna()

        # save dir
        RelativeDataDir = relativeDir / f"{qt}.csv"
        if not (RelativeDataDir).exists():
            # 1) cal relative importance for Ummae predicted values
            omics = cov + [f"{qt}_Ummae"]

            RelativeImportanceData = train_data.copy()
            RelativeImportanceDataFromat = columnsFormatV1(RelativeImportanceData)
            formated_data = RelativeImportanceDataFromat.format(RelativeImportanceData)

            omics_format = [
                RelativeImportanceDataFromat.get_format_column(i) for i in omics
            ]
            qt_format = RelativeImportanceDataFromat.get_format_column(qt)
            # formated_data

            method = "lmg"

            relimp_df = (
                calc_relimp(
                    formated_data,
                    label=qt_format,
                    vars=omics_format,
                    method=method,
                    binary_y=False,
                )
                .assign(
                    Method=method,
                )
                .melt(
                    id_vars=["Method", "R2.decomp"],
                    value_vars=omics_format,
                    var_name="Omics",
                    value_name="Importance",
                )
            )

            # 3) merged data

            relimp_df = relimp_df.assign(Model=model, Label=qt)
            relimp_df["Label"] = relimp_df["Label"].replace(
                {
                    "HbA1C%": "HbA1C",
                    "LPa": "LP(a)",
                    "hs_CRP": "hs-CRP",
                    "D_Dimer": "D-Dimer",
                    "CK_MB": "CK-MB",
                }
            )
            relimp_df["Omics"] = (
                relimp_df["Omics"]
                .replace({k: v for k, v in zip(omics_format, omics)})
                .replace({omics[-1]: "Ummae"})
            )

            relimp_df.to_csv(RelativeDataDir, index=False)
        else:
            relimp_df = pd.read_csv(RelativeDataDir)

        relative_res_list.append(relimp_df)

In [20]:
total_feature_df = pd.concat(relative_res_list)
total_feature_df.reset_index(drop=True).to_feather(
    RawDataDir / f"RelativeImportance_Ummae_concat.feather"
)
total_feature_df

,Method,R2.decomp,Omics,Importance,Model,Label
0,lmg,0.154974,Age,0.015674,Ummae,HbA1C
1,lmg,0.154974,Sex,0.007351,Ummae,HbA1C
2,lmg,0.154974,Height,0.007754,Ummae,HbA1C
3,lmg,0.154974,Weight,0.007991,Ummae,HbA1C
4,lmg,0.154974,BMI,0.011507,Ummae,HbA1C
...,...,...,...,...,...,...
1,lmg,0.005032,Sex,0.357499,Ummae,Na
2,lmg,0.005032,Height,0.102120,Ummae,Na
3,lmg,0.005032,Weight,0.061146,Ummae,Na
4,lmg,0.005032,BMI,0.019032,Ummae,Na


## ICC Calculation

- `pineguines` to calculate ICC

In [21]:
from utils import *

In [22]:
##

ICC_res = []

for model, df in totalDataDict.items():
    for qt in qts:
        for center in [
            "AZ-TR",
            "AZ-EV1",
            "AZ-EV2",
            "DX-EV",
        ]:
            model_col = f"{qt}_{model}"
            # chekc col in data
            to_cal_data = (
                totalDataDict[model]
                .query("DataSetName == @center and type not in @exclude_list ")
                .copy()
            )
            drop_no_multi_repeat_test_eid = (
                to_cal_data["eid"].value_counts().loc[lambda x: x > 1].index.tolist()
            )
            if qt not in to_cal_data.columns or model_col not in to_cal_data.columns:
                print(f"{qt} or {model_col} not in data at {center}")
                continue
            else:
                print(f"Start to calculate ICC for {qt} and {model_col} at {center}")
            df = to_cal_data.copy()[[model_col, "eid"]].dropna()

            if df.shape[0] == 0:
                print(f"No data for {qt} and {model_col} at {center}")
                continue

            results_single = (
                sample_icc_optimized(
                    df, test_name_col="eid", prob_col=model_col, k=10, M=2
                )
                .query("Type == 'ICC2'")
                .assign(Feature=qt, Model=model, Center=center)
            )

            ICC_res.append(results_single)

Start to calculate ICC for AG and AG_Ummae at AZ-TR
Start to calculate ICC for AG and AG_Ummae at AZ-EV1
Start to calculate ICC for AG and AG_Ummae at AZ-EV2
Start to calculate ICC for AG and AG_Ummae at DX-EV
Start to calculate ICC for ALP and ALP_Ummae at AZ-TR
Start to calculate ICC for ALP and ALP_Ummae at AZ-EV1
Start to calculate ICC for ALP and ALP_Ummae at AZ-EV2
Start to calculate ICC for ALP and ALP_Ummae at DX-EV
Start to calculate ICC for ALT and ALT_Ummae at AZ-TR
Start to calculate ICC for ALT and ALT_Ummae at AZ-EV1
Start to calculate ICC for ALT and ALT_Ummae at AZ-EV2
Start to calculate ICC for ALT and ALT_Ummae at DX-EV
Start to calculate ICC for APTT and APTT_Ummae at AZ-TR
Start to calculate ICC for APTT and APTT_Ummae at AZ-EV1
Start to calculate ICC for APTT and APTT_Ummae at AZ-EV2
Start to calculate ICC for APTT and APTT_Ummae at DX-EV
Start to calculate ICC for AST and AST_Ummae at AZ-TR
Start to calculate ICC for AST and AST_Ummae at AZ-EV1
Start to calculate 

In [25]:
ICC_res_df = pd.concat(ICC_res).reset_index(drop=True)
ICC_res_df.to_feather(RawDataDir / "ICC_res.feather")
ICC_res_df.to_csv(RawDataDir / "ICC_res.csv")

## Label efficiency
- For this results is precumputed with different sample sizes of training data
- This codes are used to calculate the performance of different sample sizes of training data

In [32]:
from collections import defaultdict

ClinicalLabels = pd.read_feather("Raw/ClinicalLabels.feather")
ClinicalLabels

,eid,DataSetName,AG,ALP,ALT,APTT,AST,Alb,BMI,BNP,...,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex,Stenosis
0,f412dfebe79027cff3ec509fb96cf74f,DX-EV,NaN,81.0,41.0,32.9,26.0,47.9,23.671254,45.0,...,7.50,7.66,75.0,1.52,NaN,NaN,NaN,60.0,Male,1.0
1,d2b336e0e1a1ea5c15f5cec828589dac,DX-EV,NaN,96.0,19.0,33.3,21.0,45.2,17.146776,50.0,...,4.80,5.37,45.0,1.10,NaN,NaN,NaN,69.0,Female,0.0
2,fdfe3a7ba64150efdfeb89d83cb1ccc0,DX-EV,NaN,54.0,15.0,32.6,16.0,42.3,27.458654,252.0,...,6.10,6.58,87.0,1.19,NaN,NaN,NaN,64.0,Male,0.0
3,37aba8afa41165a0ad0675b7055d227e,DX-EV,NaN,62.0,13.0,35.0,14.0,34.6,23.120624,97.0,...,4.70,7.65,70.0,0.67,NaN,NaN,NaN,74.0,Male,1.0
4,118f24f5284b781e354c37409858f59a,DX-EV,NaN,66.0,20.0,37.4,16.0,45.2,28.577961,61.0,...,4.60,5.36,75.0,1.40,NaN,NaN,NaN,60.0,Female,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9765,1f5b5321a421d4a566f6c41262ba774e,APP-EV,1.43,95.4,15.3,NaN,18.6,42.4,19.948060,NaN,...,4.77,NaN,53.0,0.30,NaN,1.72,0.41,60.0,Female,NaN
9766,234d3b32bb21791791da8741b6dd072b,APP-EV,0.98,100.5,23.3,NaN,28.8,NaN,27.942143,NaN,...,5.99,NaN,68.0,5.40,NaN,0.86,0.17,72.0,Female,NaN
9767,9b83d1551a89149543cdd635f8f2690c,APP-EV,1.38,NaN,14.9,NaN,21.9,NaN,19.628265,NaN,...,5.36,NaN,49.0,0.30,NaN,2.73,NaN,59.0,Female,NaN
9768,e0b7bca81e873378e05ddf98f369d4ed,APP-EV,1.53,NaN,28.9,NaN,26.2,NaN,32.466181,NaN,...,5.43,NaN,78.0,4.60,NaN,2.68,NaN,69.0,Female,NaN


In [47]:
data = pd.read_feather("Raw/LabelEfficiency.feather").merge(
    ClinicalLabels[["eid", "DataSetName"]],
)
data

,eid,value,model,trait,percentage,DataSetName
0,db31bc90c7b97b8f741fee9fad0d515a,77.229553,Ummae,ALP,0.1,AZ-TR
1,db31bc90c7b97b8f741fee9fad0d515a,76.765625,Ummae,ALP,0.1,AZ-TR
2,db31bc90c7b97b8f741fee9fad0d515a,76.055672,Ummae,ALP,0.1,AZ-TR
3,db31bc90c7b97b8f741fee9fad0d515a,76.024742,Ummae,ALP,0.1,AZ-TR
4,db31bc90c7b97b8f741fee9fad0d515a,76.984940,Ummae,ALP,0.1,AZ-TR
...,...,...,...,...,...,...
37161667,cf5694732dc5ecc84cd27ce37e083044,84.944458,ResNet18,BNP,0.5,AZ-EV1
37161668,cf5694732dc5ecc84cd27ce37e083044,75.056789,ResNet18,BNP,0.5,AZ-EV1
37161669,cf5694732dc5ecc84cd27ce37e083044,97.238092,ResNet18,BNP,0.5,AZ-EV1
37161670,cf5694732dc5ecc84cd27ce37e083044,84.881302,ResNet18,BNP,0.5,AZ-EV1


In [51]:
data.query("trait == 'Sex'")

,eid,value,model,trait,percentage,DataSetName


In [50]:
saveRawDatacsv = RawDataDir / "labelefficiency.csv"

if not saveRawDatacsv.exists():

    metrics_dict = defaultdict(list)

    for idx_tuple, sub_df in data.groupby(
        ["trait", "percentage", "model", "DataSetName"]
    ):
        traitName, ratio, model, center = idx_tuple
        if traitName not in ClinicalLabels.columns:
            continue

        print(f"{traitName} {ratio} {model} {center}")
        to_cal_df = (
            sub_df[["eid", "value"]]
            .copy()
            .dropna()
            .merge(ClinicalLabels[["eid", traitName]])
        )

        to_cal_df = to_cal_df.groupby("eid").mean().reset_index(drop=False)
        if len(to_cal_df) == 0:
            continue
        metrics_type = "qt"

        metrics = cal_corr_v3(
            to_cal_df,
            x="value",
            y=traitName,
        )

        metrics["Trait"] = traitName
        metrics["Ratio"] = ratio
        metrics["Model"] = model
        metrics["Center"] = center
        metrics["N"] = len(to_cal_df)

        metrics_dict[metrics_type].append(metrics)

    qt_df = pd.concat(metrics_dict["qt"]).query("Ratio !=1")

    from ppp_prediction.corr_v3 import generate_multipletests_result

    qt_df_fdr = (
        generate_multipletests_result(qt_df, method="fdr_bh")
        .sort_values("r2_score", ascending=False)[
            [
                "Model",
                "Trait",
                "Center",
                "Ratio",
                "pearsonr",
                "spearmanr",
                "N",
                "r2_score",
                "p_adj",
                "pvalue",
            ]
        ]
        .rename(columns={"r2_score": "R2"})
    )
    se, lci, uci = rsquareCI_v2(qt_df_fdr["R2"], qt_df_fdr["N"], 1)
    qt_df_fdr["R2_SE"] = se
    qt_df_fdr["R2_LCI"] = lci
    qt_df_fdr["R2_UCI"] = uci

    from ppp_prediction.metrics.utils import format_metrics

    add_metric = ["R2"]

    for metric in add_metric:

        qt_df_fdr[f"{metric} (95% CI)"] = format_metrics(
            qt_df_fdr[f"{metric}"],
            x_lci=qt_df_fdr[f"{metric}_LCI"],
            x_uci=qt_df_fdr[f"{metric}_UCI"],
            round=3,
            data=qt_df_fdr,
        )

    qt_df_fdr = qt_df_fdr.reset_index(drop=True)[
        [
            "Model",
            "Trait",
            "Center",
            "N",
            "Ratio",
            "spearmanr",
            "pearsonr",
            "p_adj",
            "pvalue",
            "R2",
            "R2_LCI",
            "R2_UCI",
            "R2 (95% CI)",
        ]
    ]  # remove models with less than 10 samples
    qt_df_fdr.sort_values(by=["Center", "Model", "Trait"], inplace=True)
    qt_df_fdr.to_csv(saveRawDatacsv, index=False)
else:
    qt_df_fdr = pd.read_csv(saveRawDatacsv)

AG 0.1 ResNet18 AZ-TR
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.1 ResNet18 AZ-EV1
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.1 ResNet18 AZ-EV2
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.1 ResNet18 AZTZ-EV
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=logistic
AG ~ value
AG 0.1 ResNet18 DX-EV
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=logistic
AG ~ value
AG 0.1 SwinLarge AZ-TR
threads is 4 and x is value 

/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is

AG 0.1 Ummae AZ-EV1
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.1 Ummae AZ-EV2
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.1 Ummae AZTZ-EV
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=logistic
AG ~ value
AG 0.1 Ummae DX-EV
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=logistic
AG ~ value
AG 0.2 ResNet18 AZ-TR
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.2 ResNet18 AZ-EV1
threads is 4 and x is value and y is AG,

/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=AG, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is

threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.2 SwinLarge AZTZ-EV
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=logistic
AG ~ value
AG 0.2 SwinLarge DX-EV
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=logistic
AG ~ value
AG 0.2 Ummae AZ-TR
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.2 Ummae AZ-EV1
threads is 4 and x is value and y is AG, which is str, so don't supported for multi threads
auto model selection for x=value and y=AG with model_type=glm
AG ~ value
AG 0.2 Ummae AZ-EV2
threads is 4 and x is value and y is AG, which is str, so 

...

/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=sdLDL, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=sdLDL, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")


threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdLDL 0.5 Ummae DX-EV
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=logistic
sdLDL ~ value
sdLDL 1.0 ResNet18 AZ-TR
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdLDL 1.0 ResNet18 AZ-EV1
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdLDL 1.0 ResNet18 AZ-EV2
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdLDL 1.0 ResNet18 AZTZ-EV

/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=sdLDL, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=sdLDL, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")


sdLDL 1.0 SwinLarge AZTZ-EV
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdLDL 1.0 SwinLarge DX-EV
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=logistic
sdLDL ~ value
sdLDL 1.0 Ummae AZ-TR
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdLDL 1.0 Ummae AZ-EV1
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdLDL 1.0 Ummae AZ-EV2
threads is 4 and x is value and y is sdLDL, which is str, so don't supported for multi threads
auto model selection for x=value and y=sdLDL with model_type=glm
sdLDL ~ value
sdL

/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=sdLDL, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")
/home/xutingfeng/ukb/project/ppp_prediction/ppp_prediction/corr_v3.py:658: UserWarning: n_case is 0.0 for y=sdLDL, which is less than 5
  warnings.warn(f"n_case is {ncase} for y={y}, which is less than 5")


In [52]:
fullRatio_qt_df = (
    pd.read_feather(RawDataDir / "metrics_qt.feather")
    .query("Label in @qt_df_fdr.Trait")
    .rename(columns={"Label": "Trait"})
    .assign(Ratio=1)
)
label_efficiency_qt_df = pd.concat(
    [
        qt_df_fdr,
        fullRatio_qt_df,
    ]
).reset_index(drop=True)


label_efficiency_qt_df.reset_index(drop=True).to_feather(
    RawDataDir / "label_efficiency_qt.feather"
)
label_efficiency_qt_df.to_csv(RawDataDir / "label_efficiency_qt.csv", index=False)